In [228]:
import numpy as np
import pandas as pd


In [252]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.read_csv('returns.csv', header=None)
# df = df.apply(lambda iterator: ((iterator - iterator.mean())/iterator.std()))

In [253]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.021732,0.023444,0.002508,-0.003098,0.009709,-0.000547,0.017476,0.015424,0.063718,0.015817,...,0.013942,0.002237,0.022508,0.018167,0.014880,-0.008362,0.005078,0.005232,0.017371,0.007844
1,0.005219,0.004288,0.003842,-0.009322,0.017449,0.018053,-0.003690,0.016591,0.003524,-0.001342,...,0.015743,-0.011905,0.003241,0.020038,-0.022159,-0.020699,-0.002210,-0.004673,0.005155,-0.024625
2,0.008180,0.013026,-0.003115,-0.008234,0.006754,0.001612,0.011543,-0.009042,0.018258,0.012455,...,0.020796,0.004217,0.013256,0.013724,0.011927,-0.005950,-0.002215,0.008644,0.009615,0.006550
3,-0.000808,-0.008324,0.029286,0.024115,0.003865,-0.004292,0.007273,-0.009681,-0.004138,-0.025046,...,0.012301,0.008698,0.000836,-0.025285,-0.006735,-0.007245,-0.008246,-0.010368,0.018730,-0.008992
4,0.007161,-0.001991,-0.007634,0.015634,0.008801,-0.007004,0.004792,0.007360,-0.013158,-0.003450,...,-0.017847,0.001784,-0.002702,-0.008280,0.002882,-0.001745,-0.002558,0.002031,0.002181,-0.002746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,-0.005340,-0.001371,-0.002687,0.002125,0.001843,-0.003877,-0.003474,-0.007024,-0.004037,-0.002903,...,0.010400,0.006996,0.002274,-0.005112,0.000573,-0.002977,-0.005046,-0.007297,-0.004043,-0.018258
1254,-0.011201,0.018724,0.005030,-0.007551,-0.006131,0.002271,0.021414,-0.001875,-0.008107,-0.000336,...,0.000606,0.005127,-0.001920,0.000856,-0.024889,-0.001148,-0.006174,-0.011531,-0.006203,0.009972
1255,0.017974,0.027799,0.010724,0.018162,0.012647,0.012298,0.014983,0.017163,0.008757,0.006497,...,0.026220,0.019188,0.006470,0.009840,0.018220,0.005059,0.018860,0.008895,0.011565,0.007052
1256,-0.013059,-0.007061,-0.004244,0.001812,-0.002437,-0.007353,-0.004610,-0.020232,-0.013310,-0.007457,...,-0.009237,-0.004295,-0.014767,-0.009320,-0.004865,-0.003203,-0.005246,-0.007226,-0.011614,-0.015717


In [254]:
# ai
def sample_cov(df):
    cov = np.array(df - df.mean(axis=0))
    total = np.zeros((cov.shape[1], cov.shape[1]))
    for r in range(cov.shape[0]):
        total += cov[[r], :].T @ cov[[r], :]
    return total/cov.shape[0]


sample_covariance = sample_cov(df)
np.savetxt('sample_covariance.csv', sample_covariance, delimiter=',')


In [246]:
def run(S, lam=0.00001, iterations = 600, stopping_thres = 0):
    T, N = df.shape
    J = np.eye(N)
    def assign(i,j, new):
        J[i][i] = (new)[0][0]
        J[i][j] = (new)[0][1]
        J[j][i] = (new)[1][0]
        J[j][j] = (new)[1][1]
    def obj():
        return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * np.sum(np.abs(J - np.diag(np.diag(J))))
        # return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * (sum(sum(abs(J))) - np.trace(abs(J)))
    prev_obj = obj()
    for _ in range(iterations):
        for i in range(N):
            for j in range(i+1,N):
                Jbb = J[[a for a in range(N) if not a in [i,j]], :][:, [a for a in range(N) if not a in [i,j]]]
                Jab = J[[i, j], :][:, [a for a in range(N) if not a in [i,j]]]
                Jba = J[[a for a in range(N) if not a in [i,j]], :][:, [i, j]]
                L = Jab@np.linalg.inv(Jbb)@Jba
                Saa = S[[i, j], :][:, [i, j]]
                Svv = S[j][j]
                Suu = S[i][i]
                ms = []
                try: 
                    m1 = np.linalg.inv(Saa+lam*np.array([[0, 1], [1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m1) > 0):
                        ms.append(m1)
                try: 
                    m2 = np.linalg.inv(Saa+lam*np.array([[0, -1], [-1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m2) > 0):
                        ms.append(m2)
                try: 
                    m3 = np.array([[(1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Suu), -L[0][1]],
                                [-L[1][0], (1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Svv)]])
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m3) > 0) and np.any(np.isnan(m3)):
                        ms.append(m3)
                curr_obj = obj()
                for m in ms:
                    prev_Jaa = J[[i,j],:][:,[i,j]]
                    assign(i, j, m+L)
                    new_obj = obj()
                    if new_obj>curr_obj:
                        curr_obj = new_obj
                    else:
                        assign(i, j, prev_Jaa)
        # if obj()-prev_obj < stopping_thres:
            # break
        if curr_obj-prev_obj > stopping_thres:
            prev_obj = curr_obj
            print(curr_obj)
        else:
            break
    print(np.count_nonzero(J))
    return J


In [247]:
J_test = run(sample_covariance, 0, 1)

-35.721400493425605
2500


In [249]:
new_J = run(sample_covariance, 0.00001)

-35.72307453882287
-25.127250919163355
-21.391934473550652
-19.68676703049526
-18.86886081513696
-18.446614637153957
-18.18063506685334
-17.96946735072074
-17.78279905763009
-17.613198294424524
-17.458639961906947
-17.31810427100719
-17.190757691521178
-17.075739824116642
-16.972064148617925
-16.87865669861505
-16.79446095264145
-16.71851225111518
-16.649939416512254
-16.58792824600674
-16.531690922193075
-16.48047639057113
-16.433593924037734
-16.39044593539882
-16.350542478090066
-16.313511280426333
-16.27908116340244
-16.247055329105443
-16.217287877118785
-16.189657188153067
-16.164050066224174
-16.140357847336254
-16.118463486755253
-16.098244799710645
-16.07957552990457
-16.062326318453824
-16.046368159626343
-16.031571551422488
-16.017811287926175
-16.004965910893752
-15.992919304651702
-15.981564808544663
-15.970806377583932
-15.960561421169054
-15.950759802537517
-15.941345792776456
-15.932278966233838
-15.923531322732831
-15.91508724873287
-15.90694226126751
-15.8990994624212

In [248]:
new_J1 = run(sample_covariance, 0.0001)

-35.73811721660774
-25.14432582082451
-21.411014025479048
-19.70691311745541
-18.88978764207107
-18.468178693362596
-18.20254304869918
-17.991510843814382
-17.804878907093613
-17.63524135720571
-17.480601265257334
-17.33997189556999
-17.212563093301338
-17.097474632065772
-16.993695897310456
-16.90020053798588
-16.815924425157576
-16.739897428160155
-16.67123100644445
-16.609122845818327
-16.55276037090064
-16.5014240474287
-16.454401256141725
-16.41109237025165
-16.371043412761562
-16.33386593248872
-16.29931958505034
-16.26720013459734
-16.237370581703715
-16.20969702378081
-16.184051560986845
-16.16034957745983
-16.13845628664075
-16.11824351139944
-16.099578811070035
-16.08232731534825
-16.066368499399378
-16.051566521988235
-16.037808649241775
-16.02496327616761
-16.012910562403356
-16.001537555040755
-15.99075332768446
-15.980481043592025
-15.970650817342694
-15.961203187972158
-15.95210666808264
-15.943333590611847
-15.934873023898476
-15.926715018691795
-15.91885491797818
-15.9

In [250]:
new_J2 = run(sample_covariance, 0.001)

-35.885946609529555
-25.31243480557151
-21.598555186036396
-19.90435490205255
-19.094527326715227
-18.678793497279457
-18.41599199839898
-18.206126447327275
-18.019336627082208
-17.84905007938673
-17.693180207332585
-17.55117927259862
-17.422709342165142
-17.306607144488854
-17.2016542294391
-17.107354173918274
-17.021979105138932
-16.944809240453015
-16.8750530926583
-16.81167989291435
-16.753711351874514
-16.70058373215731
-16.651783593683835
-16.60679756798197
-16.565241474688122
-16.527014988778735
-16.49163223279794
-16.458583263568595
-16.428070437950762
-16.39997758295157
-16.37411334112918
-16.350189589324764
-16.328247889429186
-16.308076382048984
-16.289432322787682
-16.272152070522537
-16.256229977554685
-16.241362757846645
-16.22753425714315
-16.214663433186896
-16.202548546623024
-16.191066462563676
-16.180148666834246
-16.169635356508188
-16.159581574429858
-16.149848148255842
-16.140449180630394
-16.131454426701804
-16.122812252829284
-16.114544257495663
-16.106599468860

In [251]:
print(np.count_nonzero(np.linalg.inv(new_J)))
print(np.count_nonzero(np.linalg.inv(new_J1)))
print(np.count_nonzero(np.linalg.inv(new_J2)))

2500
2500
2500


In [241]:
from sklearn.covariance import graphical_lasso
sample_covariance = df.cov().to_numpy()
S, J = graphical_lasso(sample_covariance, 0.00001)
print(np.count_nonzero(J)) # 1132
print(np.count_nonzero(S)) # 2500
S1, J1 = graphical_lasso(sample_covariance, 0.0001)
print(np.count_nonzero(J1)) # 538
print(np.count_nonzero(S1)) # 1942
S2, J2 = graphical_lasso(sample_covariance, 0.001)
print(np.count_nonzero(J2)) # 50
print(np.count_nonzero(S2)) # 50

1132
2500
538
1942
50
50


/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.527750245043871e-11, tolerance: 4.455419280884605e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0197277678291802e-09, tolerance: 3.202889249885121e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.562072513157378e-09, tolerance: 5.6190079697058765e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/p